# Model Selsction 모듈


- 훈련 데이터와 테스트 데이터로 분리하지 않고 머신러닝 수행

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

In [2]:
iris = load_iris()
dtc = DecisionTreeClassifier(random_state=2022)
dtc.fit(iris.data, iris.target )
dtc.score(iris.data,iris.target  )
# 결과 => 1.0 = 100% 
# 100%면 의심할 필요 있음 
# 분류 = 암기 
# 분류하는 로직을 보는것이 아닌 암기하는 과정을 보는것
# 이렇게 하면 무조건 틀림,,,, 안돼,,, 
# 반드시 학습과 데이터 세트를 분리 해주어야 함!!

1.0

학습/테스트 데이터 세트 분리 – `train_test_split()`
- `test_size`: 디폴트는 0.25
- `train_size`
- `shuffle`: 데이터를 분리하기 전에 데이터를 미리 섞을지를 결정. 디폴트는 True - `stratif`y: 학습 데이터의 y 값의 분포를 균등하게 만들어줌 (** 항상 세팅해줘야하는 옵션!!!)
- `random_state`
- 반환 값: 튜플 형태
  - 학습용 데이터의 피쳐 데이터 세트(X_train)
  - 테스트용 데이터의 피쳐 데이터 세트(X_test)
  - 학습용 데이터의 레이블 데이터 세트(y_train)
  - 테스트용 데이터의 레이블 데이터 세트(y_test)

- 교차 검증: `cross_validate()` + 평가
- 교차 검증 후 평가까지: `cross_val_score()` 

In [4]:
from sklearn.model_selection import cross_val_score # 이 자체로는 사용 잘 안함 

cross_val_score(dtc, iris.data, iris.target, scoring='accuracy', cv=5 ) # (모델, x, y , 방법, 몇겹 할건지)

array([0.96666667, 0.96666667, 0.9       , 0.93333333, 1.        ])

In [5]:
cross_val_score(dtc, iris.data, iris.target, scoring='accuracy', cv=5 ).mean()

0.9533333333333334

`GridSearchCV` 
- 교차 검증과 최적의 하이퍼 파라미터 튜닝 한번에 가능 
- 하이퍼 파라미터
 - Classifier나 Regressor에 사용되는 하이퍼 파라미터
 - 예측성능개선에도움을줌  GridSearchCV의 파라미터
 - estimator: classifier, regressor, pipeline
 -  param_grid: 튜닝을 위해 파라미터명과 사용될 여러가지 

- 파라미터값을 지정
 - scoring: 예측 성능을 측정할 평가 방법
 - cv: 교차 검증을 위한 학습/테스트 세트의 갯수
 - refit: 최적의 하이퍼 파라미터를 찾은 뒤, 입력된 estimator 객체를 재학습, 디폴트는 True

In [6]:
dtc.get_params()
# max_depth / min_samples_split / min_samples_split 가장 많이쓰는 것 
# 예시) 3,4,5,6 / 1,2 / 2,3 = 16 경우 + 교차검증=5 => 총 80

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 2022,
 'splitter': 'best'}

In [7]:
# Train/Test dataset 분리
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, 
                                                    stratify=iris.target, test_size=0.2, random_state=2022)

In [8]:
# 분류와 그에 해당하는 하이퍼 파라미터
dtc = DecisionTreeClassifier(random_state=2022)
dtc.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 2022,
 'splitter': 'best'}

In [9]:
# parameter들 dictionary 형태로 설정 
params={'max_depth':[2,3,4,5,6], 
        'min_samples_split':[2,3,4]
        }

In [10]:
from sklearn.model_selection import GridSearchCV
# param_grid의 파라미터들 -> 5개의 train/test dataset로 나누어서 테스트 수행 설정 
grid_dt = GridSearchCV(dtc, param_grid=params, scoring="accuracy", cv=5)
# param_grid의 하이퍼 파라미터들을 순차적으로 학습/평가
grid_dt.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=2022),
             param_grid={'max_depth': [2, 3, 4, 5, 6],
                         'min_samples_split': [2, 3, 4]},
             scoring='accuracy')

In [11]:
# best params
grid_dt.best_params_

{'max_depth': 3, 'min_samples_split': 2}

In [12]:
# best score 
grid_dt.best_score_

0.9916666666666666

In [13]:
# 최적의 분류기
#dtc_best = DecisionTreeClassifier(max_depth=3, min_samples_split=2, random_state=2022)
dtc_best = grid_dt.best_estimator_ # 동일한 코드. 위 코드보다 이게 더 편해

In [17]:
# 최적의 분류기로 예측 및 평가
dtc_best.score(X_test, y_test)

0.9333333333333333

`SVM`

In [19]:
from sklearn.svm import SVC

svc = SVC(random_state=2022)
svc.get_params()
# C = 1.0 -> 0.01, 0.1 등의 값도 가능 ?

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': 2022,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [21]:
params = {'C':[0.01, 0.1, 1, 10, 100]}
grid_sv = GridSearchCV(svc, params, scoring="accuracy", cv=5)

In [22]:
grid_sv.fit(X_train, y_train)
grid_sv.best_params_

{'C': 1}

In [24]:
# 범위를 좁혀가면서 파라미터를 찾음_1
params = {'C':[0.2, 0.6,  1, 4, 8]}
grid_sv = GridSearchCV(svc, params, scoring="accuracy", cv=5)
grid_sv.fit(X_train, y_train)
grid_sv.best_params_
#=> 4가 오히려 베스트!? 

{'C': 4}

In [25]:
# 범위를 좁혀가면서 파라미터를 찾음_2
params = {'C':[2,3,4,5,6]}
grid_sv = GridSearchCV(svc, params, scoring="accuracy", cv=5)
grid_sv.fit(X_train, y_train)
grid_sv.best_params_
#=> 범위 줄여도 베스트 값은 4 !

{'C': 4}

In [27]:
svc_best = grid_sv.best_estimator_
svc_best.score(X_test, y_test)

0.9666666666666667